In [1]:
import ast
import boto3
import pandas as pd

In [8]:
label_csv_david = 'D:/canopy_data/csvs/local_multi_label.csv'
label_csv_zhenya = '/Users/purgatorid/Documents/GitHub/canopy-gis/labelling/local_multi_label.csv'

label_df = pd.read_csv(label_csv_zhenya, converters={1:ast.literal_eval})

label_df.head()

,filepaths,labels
0,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
1,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
3,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]
4,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL]


# Grab Chip List

In [26]:
def s3_actions(bucket, in_path=None, out_path=None, copy_list=None, copy_dir=False, delete=False, stop_at=None, chip_list=False, chip_count=False):
    s3 = boto3.resource('s3')
    my_bucket = s3.Bucket(bucket)
    objs = []
    total_files = 0 
    

    
    if copy_dir or delete or chip_list or chip_count:
        for obj in my_bucket.objects.filter(Prefix=in_path):
            total_files += 1 
            objs.append(obj.key)
            if total_files == stop_at:
                break
        
    tot_objs = len(objs)
        
    if delete:
        for index,obj in enumerate(objs,1):
            print(f"deleting {index} of {tot_objs}", end='\r', flush=True)
            obj.delete()
            
    if chip_list:
        return objs
    
    if chip_count:
        return total_files
    
    if copy_list:
        total_copy_list = len(copy_list)

        for index,obj in enumerate(copy_list,1):
            
            print(f"copying {index} of {total_copy_list}", end='\r', flush=True)
            old_key = in_path + obj
            new_key = out_path + obj
            CopySource = {
                'Bucket': bucket,
                'Key':old_key}
            my_bucket.copy(CopySource, new_key)

In [27]:
bucket = 'canopy-production-ml'
path = 'chips/cloudfree-merge-polygons/split/train/'

chips = s3_actions(bucket,in_path=path,chip_list=True)

## make dataframe of s3 filepaths

In [7]:
s3 = boto3.resource('s3')

bucket_name = 'canopy-production-ml'

pc_bucket = s3.Bucket(bucket_name)

print(pc_bucket)

s3.Bucket(name='canopy-production-ml')


In [8]:
train_chips = []

val_chips = []

train_uri = 'chips/cloudfree-merge-polygons/split/train/'

val_uri = 'chips/cloudfree-merge-polygons/split/val/'

for obj in pc_bucket.objects.all():
    #if train_uri in obj.key:
    #    train_chips.append(obj.key)
    if val_uri in obj.key:
        val_chips.append(obj.key)

In [9]:
len(val_chips)

6795

In [10]:
val_chips[0]

'chips/cloudfree-merge-polygons/split/val/1/1_1000_1000.tif'

In [11]:
df_val_s3 = pd.DataFrame(data={'s3_paths': val_chips})

df_val_s3.head()

,s3_paths
0,chips/cloudfree-merge-polygons/split/val/1/1_1...
1,chips/cloudfree-merge-polygons/split/val/1/1_1...
2,chips/cloudfree-merge-polygons/split/val/1/1_1...
3,chips/cloudfree-merge-polygons/split/val/1/1_1...
4,chips/cloudfree-merge-polygons/split/val/1/1_1...


## merge dataframes

In [12]:
test_1 = pd.DataFrame(data={'col1': [1,2,3], 'col2': [4,5,6]})

test_2 = pd.DataFrame(data={'col2': [5,6,4], 'col3': [8,9,10]})

test_1.merge(test_2, on='col2')

,col1,col2,col3
0,1,4,10
1,2,5,8
2,3,6,9


In [13]:
label_df.loc[0, 'filepaths']

'/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Chips/misha_polygons_cloudfreemerge/yes/ISL/100/59/59_800_2800.tif'

In [14]:
df_val_s3.loc[0, 's3_paths']

'chips/cloudfree-merge-polygons/split/val/1/1_1000_1000.tif'

In [15]:
def get_filename(path):
    return path.split('/')[-1]

label_df['filenames'] = label_df['filepaths'].apply(get_filename)
df_val_s3['filenames'] = df_val_s3['s3_paths'].apply(get_filename)

In [16]:
label_df.head()

,filepaths,labels,lengths,filenames
0,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],1,59_800_2800.tif
1,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],1,59_900_2800.tif
2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],1,59_900_2900.tif
3,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],1,59_1000_2900.tif
4,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],1,59_1000_3000.tif


In [17]:
df_val_s3.head()

,s3_paths,filenames
0,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1000.tif
1,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1100.tif
2,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1200.tif
3,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1300.tif
4,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1400.tif


In [18]:
merge_df_val = df_val_s3.merge(label_df, on='filenames')

merge_df_val.head()

,s3_paths,filenames,filepaths,labels,lengths
0,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1000.tif,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,"[ISL, Rainforest]",2
1,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1100.tif,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Rainforest],1
2,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1200.tif,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Rainforest],1
3,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1300.tif,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Rainforest],1
4,chips/cloudfree-merge-polygons/split/val/1/1_1...,1_1000_1400.tif,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Rainforest],1


In [19]:
assert len(merge_df_val) == len(val_chips)

In [20]:
from sklearn.preprocessing import MultiLabelBinarizer


mlb = MultiLabelBinarizer()

val_ohe = mlb.fit_transform(merge_df_val['labels'])

val_ohe

array([[0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [22]:
mlb.classes_

array(['Habitation', 'ISL', 'Industrial_agriculture', 'Mining',
       'Rainforest', 'River', 'Roads', 'Savannah', 'Shifting_cultivation',
       'Water'], dtype=object)

In [21]:
list(val_ohe)[0]

array([0, 1, 0, 0, 1, 0, 0, 0, 0, 0])

In [23]:
len(val_ohe)

6795

In [26]:
with open('val.lst', 'w') as lst:
    for i in range(len(merge_df_val)):
        lst.write(str(i) + '\t')
        for label in list(val_ohe)[i]:
            lst.write(str(label) + '\t')
        lst.write(merge_df_val.loc[i, 's3_paths'])
        lst.write('\n')

## the function

In [16]:
label_df

,filepaths,labels,filenames
0,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],59_800_2800.tif
1,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],59_900_2800.tif
2,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],59_900_2900.tif
3,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],59_1000_2900.tif
4,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[ISL],59_1000_3000.tif
...,...,...,...
41766,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Industrial_agriculture],49_2300_2800.tif
41767,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Industrial_agriculture],49_2300_2900.tif
41768,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Industrial_agriculture],49_2300_3000.tif
41769,/Volumes/Lacie/zhenyadata/Project_Canopy_Data/...,[Industrial_agriculture],49_2400_2500.tif


In [36]:
from sklearn.preprocessing import MultiLabelBinarizer


def make_lst_file(label_df, chips, lst_path):
    s3_df = pd.DataFrame(data={'s3_paths': chips})
    
    def get_filename(path):
        return path.split('/')[-1]

    label_df['filenames'] = label_df['filepaths'].apply(get_filename)
    s3_df['filenames'] = s3_df['s3_paths'].apply(get_filename)
    
    merge_df = s3_df.merge(label_df, on='filenames')
    
    mlb = MultiLabelBinarizer()
    ohe = mlb.fit_transform(merge_df['labels'])

    with open(lst_path, 'w') as lst:
        for i in range(len(merge_df)):
            lst.write(str(i) + '\t')
            for label in list(ohe)[i]:
                lst.write(str(label) + '\t')
            lst.write(merge_df.loc[i, 's3_paths'])
            lst.write('\n')
            
    return mlb.classes_

In [37]:
make_lst_file(label_df, chips, 'train_1.lst')

array(['Habitation', 'ISL', 'Industrial_agriculture', 'Mining',
       'Rainforest', 'River', 'Roads', 'Savannah', 'Shifting_cultivation',
       'Water'], dtype=object)